In [ ]:
%reset -f

In [ ]:
import tensorflow as tf
import pyreadr
from tensorflow import keras
from keras.applications.resnet_v2 import ResNet50V2, ResNet152V2, preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [ ]:
result = pyreadr.read_r('df_002.rda')
result2 = pyreadr.read_r('df2_002.rda')
result3 = pyreadr.read_r('c3_002.rda')
result4 = pyreadr.read_r('df_max_002.rda')
result5 = pyreadr.read_r('df_min_002.rda')
result6 = pyreadr.read_r('n_002.rda')

print(result.keys())
print(result2.keys())
print(result3.keys())
print(result4.keys())
print(result5.keys())
print(result6.keys())

In [ ]:
df = result["df"]
df2 = result2["df2"]
c3 = result3["c3"]
df_max = result4["df_max"]
df_min = result5["df_min"]
n = result6["n"]
print(df_max)
print(df_min)

In [ ]:
r, n_road = c3.shape
print(n_road)

In [ ]:
n_lookback = 24
n_sample = int(len(df)/(n_road) - n_lookback)
n_time = 1
channel = 3

In [ ]:
import numpy as np
  
X_mat = np.zeros([n_sample, n_road, n_lookback, channel], dtype = float)
Y_mat = np.zeros([n_sample, n_road], dtype = float)
# print("Matrix b : \n", b)
print(X_mat.shape)
print(Y_mat.shape)
print(type(X_mat))

In [ ]:
for i in (number for number in range(n_sample)):
    for j in (number for number in range(n_lookback)):
        st = (j+i)*(n_road) + 1
        en = ((j+i)+1)*(n_road)
        st = st - 1
        en = en - 1
        X_mat[i,:,j,2] = df[st:(en+1)].to_numpy().flatten()
    
    st2 = (i+n_lookback)*(n_road)
    en2 = (i+n_lookback+1)*(n_road)
    Y_mat[i,:] = df[st2:(en2)].to_numpy().flatten()


In [ ]:
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import numpy as np
import tensorflow as tf
from tensorflow import keras

epochs = [5, 10]
input = Input(shape=(n_road, n_lookback, 3))

In [ ]:
for p in (number for number in range(len(epochs))):
    print(epochs[p])
    base_model = ResNet152V2(input_tensor = input, include_top = False, weights = "imagenet")
    x = base_model(input, training=False)
    model = Sequential([
      base_model,
      keras.layers.GlobalAveragePooling2D(),
#       keras.layers.Dense(128, activation = 'relu'),
#       keras.layers.Dense(128, activation = 'relu'),
#       keras.layers.Dense(128, activation = 'relu'),
      keras.layers.Flatten(),
      keras.layers.Dense(n_road)
    ])
    # model.summary()
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss='mean_absolute_error',)

    model.fit(X_mat, Y_mat, epochs = epochs[p]) 
    n_road2 = n_road
    n_lookback2 = n_lookback
    n_sample2 = int(len(df2)/(n_road2) - n_lookback2)
    n_time2 = 1
    X_mat2 = np.zeros([n_sample2, n_road2, n_lookback2, channel], dtype = float)
    Y_mat2 = np.zeros([n_sample2, n_road2], dtype = float)
  
    # print("Matrix b : \n", b)
    print(X_mat2.shape)
    print(Y_mat2.shape)
    print(type(X_mat2))
    
    for k in (number for number in range(n_sample2)):
        for l in (number for number in range(n_lookback2)):
    #         if j == 0 and i == 0:
    #             st = 0
    #             en = ((j+i)+1)*(n_road-1)
    #         else:
            st3 = (l+k)*(n_road2) + 1
            en3 = ((l+k)+1)*(n_road2)
            st3 = st3 - 1
            en3 = en3 - 1

            X_mat2[k,:,l,2] = df2[st3:(en3+1)].to_numpy().flatten()

        st4 = (k+n_lookback2)*(n_road2)
        en4 = (k+n_lookback2+1)*(n_road2)
        Y_mat2[k,:] = df[st4:(en4)].to_numpy().flatten()
        
    y_pred = model(X_mat2).numpy()
        
    preds2 = (y_pred*(df_max.to_numpy().flatten() - df_min.to_numpy().flatten())) + df_min.to_numpy().flatten()
        

    Y_mat2 = (Y_mat2*(df_max.to_numpy().flatten() - df_min.to_numpy().flatten())) + df_min.to_numpy().flatten()

    
    # pip install sklearn
    from sklearn.metrics import mean_absolute_error
    mae = mean_absolute_error(preds2, Y_mat2)
    
    preds2max = preds2.max()
    preds2min = preds2.min()
    
    tests = ["model_002_152_nadd" + str(epochs[p]) + ".pkl"]
    import pickle
    with open(tests[0], 'wb') as f:  
        pickle.dump([mae, preds2max, preds2min], f)

    # Getting back the objects:
    with open(tests[0], 'rb') as f:  # Python 3: open(..., 'rb')
        mae, preds2max, preds2min = pickle.load(f)
    
    tests = ["model/model_002_152_nadd" + str(epochs[p]) + ".h5"]
    model.save(tests[0])
    print(mae, preds2max, preds2min)
